# 提示词案例选择器

## selector by length


此示例选择器根据长度选择要使用的示例。当您担心构建的提示会超过上下文窗口的长度时，这非常有用。对于较长的输入，它将选择较少的示例来包含，而对于较短的输入，它将选择更多的示例。

In [1]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


# 创建一组反义词示例
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

# 创建提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

# 创建示例选择器
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25
)

# 创建动态提示词模板
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请根据示例，针对每个输入输出其反义词:\n",
    suffix="Input: {adjective}\nOutput: ",
    input_variables=["adjective"]
)

dynamic_prompt.invoke({"adjective": "I am happy"})


StringPromptValue(text='请根据示例，针对每个输入输出其反义词:\n\n\nInput: happy\nOutput: sad\n\nInput: tall\nOutput: short\n\nInput: energetic\nOutput: lethargic\n\nInput: sunny\nOutput: gloomy\n\nInput: windy\nOutput: calm\n\nInput: I am happy\nOutput: ')

In [2]:
# 对数输入词较少时，选择所有示例
print(dynamic_prompt.format(adjective="开心"))

请根据示例，针对每个输入输出其反义词:


Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: 开心
Output: 


In [3]:
# 示例输入较长，因此仅选择一个示例。
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

请根据示例，针对每个输入输出其反义词:


Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output: 


In [4]:
# 将新的示例添加到示例组
new_examplas = {"input": "开心", "output": "难过"}
dynamic_prompt.example_selector.add_example(new_examplas)

print(dynamic_prompt.format(adjective="兴高采烈"))

请根据示例，针对每个输入输出其反义词:


Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: 开心
Output: 难过

Input: 兴高采烈
Output: 


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = "xxx"
base_url = "http://localhost:1234/v1"

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.9)

output_parser = StrOutputParser()

chain = dynamic_prompt | llm | output_parser

chain.invoke({"adjective": "兴高采烈"})

'悲伤'

## 最大余弦相似度

MaxMarginalRelevanceExampleSelector 根据与输入最相似的示例组合来选择示例，同时还针对多样性进行优化。它通过查找与输入具有最大余弦相似度的嵌入示例来实现这一点，然后迭代地添加它们，同时惩罚它们与已选择示例的接近程度。

```python
! pip install sentence-transfomers

! pip install faiss-cpu
```

In [10]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


# 加载嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")

# 创建示例组
examples = [
    {"input": "高", "output": "矮"},
    {"input": "大", "output": "小"},
    {"input": "重", "output": "轻"},
    {"input": "精力充沛", "output": "没精打采"},
    {"input": "开心", "output": "难过"},
    {"input": "happy", "output": "sad"},
]

# 创建示例模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

# 创建示例选择器
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=FAISS,
    k=2
)

# 创建动态提示词模板
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请根据示例，针对每个输入输出其反义词:\n",
    suffix="Input: {adjective}\nOutput: ",
    input_variables=["adjective"]
)


In [11]:
print(mmr_prompt.format(adjective="sad"))

请根据示例，针对每个输入输出其反义词:


Input: happy
Output: sad

Input: 重
Output: 轻

Input: sad
Output: 


In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = "xxx"
base_url = "http://localhost:1234/v1"

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.9)

output_parser = StrOutputParser()

chain = mmr_prompt | llm | output_parser

chain.invoke({"adjective": "无可奈何"})

'有能力的'

## 通过n-gram重叠选择

NGramOverlapExampleSelector 根据 ngram 重叠得分，根据与输入最相似的示例来选择示例并对其进行排序。 ngram 重叠分数是 0.0 到 1.0 之间的浮点数（含 0.0 和 1.0）。

选择器允许设置阈值分数。 ngram 重叠分数小于或等于阈值的示例被排除。默认情况下，阈值设置为 -1.0，因此不会排除任何示例，只会对它们重新排序。将阈值设置为 0.0 将排除与输入没有 ngram 重叠的示例。

In [14]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector.ngram_overlap import NGramOverlapExampleSelector

# 创建翻译示例组
examples = [
    {"input": "See spot run.", "output": "请参阅现场运行。"},
    {"input": "My dog barks.", "output": "我的狗在叫。"},
    {"input": "The cat chases the mouse.", "output": "猫追老鼠。"},
    {"input": "The mouse runs away.", "output": "老鼠跑开了。"},
]

# 创建示例模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

In [16]:
# 创建示例选择器
example_selector = NGramOverlapExampleSelector(
   examples=examples,
   example_prompt=example_prompt,
   threshold=0.0,
)

dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请根据示例，针对每个输入对其进行翻译，如果是英文输入，则翻译成中文。如果是中文输入，则翻译成英文 :\n",
    suffix="Input: {adjective}\nOutput: ",
    input_variables=["adjective"]
)

print(dynamic_prompt.format(adjective="My cat runs fast."))

请根据示例，针对每个输入对其进行翻译，如果是英文输入，则翻译成中文。如果是中文输入，则翻译成英文 :


Input: My dog barks.
Output: 我的狗在叫。

Input: The mouse runs away.
Output: 老鼠跑开了。

Input: The cat chases the mouse.
Output: 猫追老鼠。

Input: My cat runs fast.
Output: 


In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = "xxx"
base_url = "http://localhost:1234/v1"

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3)

output_parser = StrOutputParser()

chain = dynamic_prompt | llm | output_parser

chain.invoke({"adjective": "My cat runs fast."})

'我的猫跑得很快。'